In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

configs = [
    dict(symbol='XAUUSD', timeframe='H1', direction='buy',
         train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
         test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
         search_type='clusters', subtype='simple',
         n_trials=500, n_models=5, n_jobs=-1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='buy',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='clusters', subtype='advanced',
    #      n_trials=500, n_models=5, n_jobs=-1),
    dict(symbol='XAUUSD', timeframe='H1', direction='buy',
         train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
         test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
         search_type='markov', subtype='simple',
         n_trials=500, n_models=5, n_jobs=-1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='buy',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='markov', subtype='advanced',
    #      n_trials=500, n_models=5, n_jobs=-1),
]

# Calcular núcleos por configuración
total_cores = os.cpu_count()
cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["n_jobs"] = cores_per_config
    cfg["tag"]    = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['search_type']} {cfg['subtype']}"

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()
    return s.all_results

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado: {f.result()}")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 buy markov simple] modelo 0 trial 1/500 best=-1.0000 avg=  6.89s[XAUUSD H1 buy clusters simple] modelo 0 trial 1/500 best=-1.0000 avg=  6.89s



Model is not converging.  Current: -7984.607779123666 is not greater than -7945.244337385408. Delta is -39.36344173825819


[XAUUSD H1 buy markov simple] modelo 0 trial 2/500 best=1.5861 avg= 23.58s
[XAUUSD H1 buy clusters simple] modelo 0 trial 2/500 best=3.4121 avg= 71.74s
[XAUUSD H1 buy clusters simple] modelo 0 trial 3/500 best=3.4121 avg= 47.92s
[XAUUSD H1 buy clusters simple] modelo 0 trial 4/500 best=3.4121 avg= 37.66s
[XAUUSD H1 buy markov simple] modelo 0 trial 3/500 best=3.1399 avg= 64.44s
[XAUUSD H1 buy markov simple] modelo 0 trial 4/500 best=3.1399 avg= 73.13s
[XAUUSD H1 buy markov simple] modelo 0 trial 5/500 best=3.1399 avg= 58.68s
[XAUUSD H1 buy markov simple] modelo 0 trial 6/500 best=3.1399 avg= 49.26s
[XAUUSD H1 buy markov simple] modelo 0 trial 7/500 best=3.1399 avg= 42.24s
[XAUUSD H1 buy markov simple] modelo 0 trial 8/500 best=3.1399 avg= 37.00s
